In [ ]:
# | default_exp soil_utils

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import os
import operator
import numpy as np
from math import pi
import pandas as pd
import pandera as pa
from typing import Dict
from pathlib import Path, PosixPath
from pydantic import ValidationError

# from pandera.typing import Series, DataFrame
from collections import OrderedDict, defaultdict
from pysureau.pysureau_utils import dict_to_csv
from pysureau.parameter_validators import (
    SoilParameterValidatorCampbell,
    SoilParameterValidatorVg,
)

In [ ]:
# | export
def compute_b(
    lv: float,  # length of fine root per unit volume
) -> float:
    "Calculate b used to compute the B of the Gardnar-Cowen model"

    return 1 / np.sqrt(pi * lv)

In [ ]:
# | export
def compute_b_gc(
    la: float,  # Unknown parameter definition
    b: float,  # Unknown parameter definition
    root_radius: float,  # Calculated using the `compute_b` function
) -> float:
    "Calculate B Gardner cowen thhe scaling factor for soil conductance"

    return la * 2 * pi / np.log(b / root_radius)

In [ ]:
# | export
def compute_k_soil(
    rew: float,  # Unknown parameter definition
    i_vg: float,  # Unknown parameter definition
    n_vg: float,  # Unknown parameter definition
    k_sat_vg: float,  # Unknown parameter definition
    b_gc: float,  # Calculated using the `compute_b_gc` function
) -> float:
    # Create empty dict for storing params --------------------------------------
    k_soil_parameters = collections.defaultdict(list)

    # Compute k_soil ------------------------------------------------------------
    m = 1 - (1 / n_vg)

    k_soil = k_sat_vg * rew ** (i_vg) * (1 - (1 - rew ** (1 / m)) ** m) ** 2

    k_soil_gc = 1000 * b_gc * k_soil

    # Append to dictionary ------------------------------------------------------
    k_soil_parameters['k_soil'] = k_soil
    k_soil_parameters['k_soil_gc'] = k_soil_gc

    return k_soil_parameters

In [ ]:
# | export
def compute_k_soil_camp(
    sws: float,  # Unknown parameter definition
    tsc: float,  # Unknown parameter definition
    b_camp: float,  # Unknown parameter definition
    k_sat_campbell: float,  # Unknown parameter definition
) -> float:
    return k_sat_campbell * (sws / tsc) ** (-b_camp * 2 + 2)

In [ ]:
# | export
def compute_p_soil(
    rew: float,  # Unknown parameter definition
    alpha_vg: float,  # Unknown parameter definition
    n_vg: float,  # Unknown parameter definition
) -> float:
    m = 1 - (1 / n_vg)

    # diviser par 10000 pour passer de cm à MPa
    return -1 * ((((1 / rew) ** (1 / m)) - 1) ** (1 / n_vg)) / alpha_vg / 10000

In [ ]:
# | export
def compute_p_soil_camp(
    sws: float,  # Unknown parameter definition
    tsc: float,  # Unknown parameter definition
    b_camp: float,  # Unknown parameter definition
    psie: float,  # Unknown parameter definition
) -> float:
    return -1 * (psie * ((sws / tsc) ** -b_camp))

In [ ]:
# | export
def compute_theta_at_given_p_soil(
    psi_target: float,  # Unknown parameter definition
    theta_res: float,  # Unknown parameter definition
    theta_sat: float,  # Unknown parameter definition
    alpha_vg: float,  # Unknown parameter definition
    n_vg: float,  # Unknown parameter definition
) -> float:
    # Assert that values are positive.
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one
    # type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(psi_target),
        0,
        err_msg='\nError: psi_target values must be greater than 0\n',
    )

    return theta_res + (theta_sat - theta_res) / (
        1 + (alpha_vg * psi_target * 10000) ** n_vg
    ) ** (1 - 1 / n_vg)

#### __Example: Compute theta parameter__

In [ ]:
compute_theta_at_given_p_soil(
    psi_target=2, theta_res=3, theta_sat=4, alpha_vg=5, n_vg=1.1
)

3.316227675107904

In [ ]:
compute_theta_at_given_p_soil(
    psi_target=np.array([1.5, 2]),
    theta_res=np.array([3, -2]),
    theta_sat=np.array([-6, -7]),
    alpha_vg=np.array([9.02, 2.9]),
    n_vg=np.array([-1.5, 0.001]),
)

array([-5.99999970e+000, -6.51316634e+303])

In [ ]:
# | export
def compute_theta_at_given_p_soil_camp(
    theta_sat: float,  # Unknown parameter definition
    psi_target: float,  # Unknown parameter definition
    psie: float,  # Unknown parameter definition
    b_camp: float,  # Unknown parameter definition
) -> float:
    # Assert that values are negative.
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one
    # type

    return theta_sat * (psi_target / psie) ** (1 / b_camp)

#### __Example: Compute theta parameter for Campbell formulation__

In [ ]:
compute_theta_at_given_p_soil_camp(
    psi_target=-1.5, theta_sat=0.39, psie=-0.025, b_camp=-4.0
)

0.14012860366560284

In [ ]:
compute_theta_at_given_p_soil_camp(
    psi_target=np.array([-1.5, -2]),
    theta_sat=np.array([0.70]),
    psie=np.array([-0.025, -4]),
    b_camp=np.array([-4.0, -6]),
)

array([0.25151288, 0.78572343])

In [ ]:
# | export


def create_empty_soil_parameter_files(
    path: Path,  # Path to the folder where the parameter files will be saved. If set to None then the files will be saved at the current working directory
) -> Dict:  # Return two dictionary files for user input
    "Function for creating the CSV templates necessary for the soil parameters"

    # Assert parameters ---------------------------------------------------------
    assert isinstance(path, str) | isinstance(path, PosixPath), (
        f'Input path must be a str, not a {type(path).__name__}'
    )

    # Convert string to Path if provided ----------------------------------------
    path = Path(path)
    if os.path.exists(path):
        # Soil parameters for van Genuchten pedo transfer function
        soil_params_vg = {
            'alpha_vg': 'NA',
            'g_soil_0': 'NA',
            'i_vg': 'NA',
            'ksat_vg': 'NA',
            'n_vg': 'NA',
            'offset_psoil': 'NA',
            'pedo_transfer_formulation': 'vg',
            'psie': 'NA',
            'psie_vg': 'NA',
            'psoil_at_field_capacity': 'NA',
            'psoil_at_wilting_point': 1.5,
            'residual_capacity_vg': 0.098,
            'rfc_1': 'NA',
            'rfc_2': 'NA',
            'rfc_3': 'NA',
            'reset_swc': False,
            'soil_depth_1': 'NA',
            'soil_depth_2': 'NA',
            'soil_depth_3': 'NA',
            'soil_evap': True,
            'saturation_capacity_vg': 'NA',
            'soil_formulation': 'vg',
            'water_soil_transfer': True,
        }

        # Soil parameters for Campbell pedo transfer funtion
        soil_params_campbell = {
            'b_camp': 'NA',
            'g_soil_0': 'NA',
            'ksat_campbell': 'NA',
            'offset_psoil': 'NA',
            'pedo_transfer_formulation': 'campbell',
            'psie': 'NA',
            'psie_camp': -0.025,
            'psoil_at_field_capacity': 'NA',
            'psoil_at_wilting_point': 1.5,
            'residual_capacity_campbell': 0.098,
            'rfc_1': 'NA',
            'rfc_2': 'NA',
            'rfc_3': 'NA',
            'reset_swc': False,
            'saturation_capacity_campbell': 'NA',
            'soil_depth_1': 'NA',
            'soil_depth_2': 'NA',
            'soil_depth_3': 'NA',
            'soil_evap': True,
            'soil_formulation': 'campbell',
            'water_soil_transfer': True,
        }

        # Write CSV files for vg and campbell -----------------------------------
        dict_to_csv(
            dictionary=soil_params_vg,
            path=path,
            filename='soil_parameters_vg.csv',
        )

        dict_to_csv(
            dictionary=soil_params_campbell,
            path=path,
            filename='soil_parameters_campbell.csv',
        )

    else:
        raise ValueError('Failed creating empty parameter files')

In [ ]:
# | export
def read_soil_file(
    file_path: Path,  # Path to the sureau_parameter_files folder containing the csv files with parameter values i.e path/to/sureau_parameter_files/file_name.csv
    sep: str = ',',  # CSV file separator can be ',' or ';'
) -> Dict:  # Dictionary with soil parameters
    "Function for reading a CSV file containing soil parameters information"

    # Assert parameters ---------------------------------------------------------

    # Make sure that sureau_parameter_files folder exist

    # Make sure the file_path exist
    assert os.path.exists(file_path), (
        f'sureau_parameter_files folder not found at {file_path}. Save soil parameter files within sureau_parameter_files'
    )

    # Read and validate dataframe -----------------------------------------------

    # Read CSV
    soil_data = pd.read_csv(file_path, sep=sep, header=0)

    # Validate the column names of soil data are parameter_name, parameter_value
    # To correctly transform the the CSV file as dict later
    if soil_data.columns.tolist() != ['parameter_name', 'parameter_value']:
        raise ValueError(
            'Column names in soil parameter file must be called parameter_name and parameter_value'
        )

    # Transform dataframe into dictionary ---------------------------------------
    soil_data_dict = soil_data.set_index('parameter_name').to_dict()[
        'parameter_value'
    ]

    # Loop over dictionary to transform the data types --------------------------
    # If this is not done all values will be considered str

    # Loop over all keys
    for each_key in soil_data_dict.keys():
        # If value is 'vg' or 'campbell' then transform to str
        if (
            soil_data_dict[each_key] == 'vg'
            or soil_data_dict[each_key] == 'campbell'
        ):
            soil_data_dict[each_key] = str(soil_data_dict[each_key])

        elif (
            soil_data_dict[each_key] == 'TRUE'
            or soil_data_dict[each_key] == 'FALSE'
            or soil_data_dict[each_key] == 'True'
            or soil_data_dict[each_key] == 'False'
        ):
            soil_data_dict[each_key] = bool(soil_data_dict[each_key])

        # Transform parameters values float
        else:
            soil_data_dict[each_key] = float(soil_data_dict[each_key])

    # Validate, raise error if soil_data_dict don't follow the Schema -----------
    if soil_data_dict['soil_formulation'] == 'campbell':
        try:
            SoilParameterValidatorCampbell.model_validate(soil_data_dict)

        except ValidationError as error:
            raise (error)

    else:
        try:
            SoilParameterValidatorVg.model_validate(soil_data_dict)

        except ValidationError as error:
            raise (error)

    # Setting common parameters for WB_soil (regardless of the options) ---------
    if soil_data_dict['pedo_transfer_formulation'] == 'vg':
        # 21 params
        params = np.array(
            [
                'alpha_vg',
                'g_soil_0,',
                'i_vg',
                'ksat_vg',
                'n_vg',
                'offset_psoil',
                'pedo_transfer_formulation',
                'psie',
                'psoil_at_field_capacity',
                'psoil_at_wilting_point',
                'residual_capacity_vg',
                'rfc_1',
                'rfc_2',
                'rfc_3',
                'reset_swcsoil_depth_1',
                'soil_depth_2',
                'soil_depth_3',
                'soil_evap',
                'saturation_capacity_vg',
                'soil_formulation',
                'water_soil_transfer',
            ],
            dtype=object,
        )

    elif soil_data_dict['pedo_transfer_formulation'] == 'campbell':
        # 12 params
        params = np.array(
            [
                'rfc_1',
                'rfc_2',
                'rfc_3',
                'soil_depth_1',
                'soil_depth_2',
                'soil_depth_3',
                'psoil_at_field_capacity',
                'g_soil_0',
                'offset_psoil',
                'pedo_transfer_formulation',
                'psieb_camp',
                'saturation_capacity_campbell',
                'ksat_campbell',
            ],
            dtype=object,
        )
    else:
        raise ValueError(
            f'Option {soil_data_dict["pedo_transfer_formulation"]} not recognized. Set pedo_transfer_function to either "vg" or "campbell"'
        )

    # Make sure there are no duplicate parameters -------------------------------
    if len(soil_data_dict.keys()) is not len(set(soil_data_dict.keys())):
        raise ValueError(
            'Parameter might be repeated several times in input soil parameter file'
        )

    # Return
    return defaultdict(list, soil_data_dict)

In [ ]:
# read_soil_file(file_path='/tmp/pysureau_project_bnXyZFMV/1_parameter_files/soil_parameters_campbell.csv')

In [ ]:
# read_soil_file(file_path='/tmp/pysureau_project_bnXyZFMV/1_parameter_files/soil_parameters_vg.csv')

In [ ]:
# | export


def convert_vwc_to_sws(
    vwc_x: float,  # Volumetric Water Content m3.m-3
    layer_thickness: float,  # Soil layer thickness in meters?
    rfc: int = 0,  # Rock Fragment Content (%)
) -> float:  # Soil Water Stock (mm)
    "Convert soil volumetric water content (m3.m-3) to water stock height (quantity as height in mm per m2 soil) by accounting for the respective layer thickness and rock fragment content. The volume of the water quantity per square metre results in the corresponding water stock height (m3 water per m2 soil as height in mm)"

    return vwc_x * (1 - (rfc / 100)) * layer_thickness * 1000

In [ ]:
convert_vwc_to_sws(1, layer_thickness=1)

1000.0

In [ ]:
# | export


def convert_sws_to_vwc(
    sws_x: float,  # Soil Water Stock (mm)
    layer_thickness: float,  # Soil layer thickness in meters?
    rfc: int = 0,  # Rock Fragment Content (%)
) -> float:  # Volumetric Water Content m3.m-3
    "Convert soil water stock (quantity as height in mm per m2 soil) to volumetric water content (m3.m-3) by accounting for the respective layer thickness and rock fragment content"

    return sws_x / ((1 - (rfc / 100)) * layer_thickness * 1000)

In [ ]:
convert_sws_to_vwc(1000, layer_thickness=1)

1.0